In [9]:
import pandas as pd
import numpy as np
import time as t
import re, os, sys
import itertools

sys.path.append(os.getcwd())

from datetime import datetime as dt
from PF_API_Shared import get_all_plants, get_plant, get_plant_attributes, get_plant_devices, Dispatch,TimeInterval
from API_Intergration import plant_tz
from src.config import SUBSCRIPTION_KEY, CUSTOMER_ID

In [2]:
plant = 'Gulf I'
start = dt(2022,9,1)
end = dt(2022,9,3)

In [11]:
dev_tags = [("Inverter", ["AC_POWER"]),
            ("Sensor Pyranometer POA", ["IRRADIANCE_POA"]),
            ("Meter",["AC_POWER"])]
#DC Current & Curtailment     
rq_tags = []

plants = get_all_plants(SUBSCRIPTION_KEY, CUSTOMER_ID)  
plant_id = plants["id"][plants["name"] == plant].values[0]
dispatch = Dispatch(SUBSCRIPTION_KEY, '2096')
devices = get_plant_devices(SUBSCRIPTION_KEY,CUSTOMER_ID,plant_id)

inv_description = devices[devices['description'].str.contains('kW')]

inv = [x for x in inv_description["description"]]

In [12]:
for dtype, tags in dev_tags:
    dtype_devices = devices[devices['type']==dtype]
    
    df = dispatch.fetch_data(dtype_devices['id'], 
                                tags, 
                                query_range=TimeInterval(starttime=start,endtime=end))
    rq_tags.append(df)
inv_data = pd.concat(rq_tags, axis=1)
data = inv_data.copy(deep=True)

ident = [id for site,type,id,name in data.columns]

In [14]:
data.columns = ident
list_compile = lambda a,b: a + '-' + b


for i in data.columns:
    if "INV" in i:
        mapper = (list(map(list_compile, ident,inv)))
df = data.filter(regex="INV")
df.columns = mapper
df.loc[:,"POA"] = data.filter(regex="MET").median(axis=1)
df.loc[:, "Meter Power"] = data.filter(regex="MTR").median(axis=1)
df


C:\Users\dpinales\AppData\Local\Temp\ipykernel_78324\2777365860.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:,"POA"] = data.filter(regex="MET").median(axis=1)
C:\Users\dpinales\AppData\Local\Temp\ipykernel_78324\2777365860.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, "Meter Power"] = data.filter(regex="MTR").median(axis=1)


,2096.05.005.BLK01.PAD01.INV01-Inverter 1.1.1 - 3018.75kW - 2500.00kW,2096.05.005.BLK01.PAD02.INV01-Inverter 1.2.1 - 3018.75kW - 2500.00kW,2096.05.005.BLK01.PAD03.INV01-Inverter 1.3.1 - 3018.75kW - 2500.00kW,2096.05.005.BLK01.PAD04.INV01-Inverter 1.4.1 - 3018.75kW - 2500.00kW,2096.05.005.BLK01.PAD05.INV01-Inverter 1.5.1 - 3018.75kW - 2500.00kW,2096.05.005.BLK01.PAD06.INV01-Inverter 1.6.1 - 3003.75kW - 2500.00kW,2096.05.005.BLK01.PAD07.INV01-Inverter 1.7.1 - 3003.75kW - 2500.00kW,2096.05.005.BLK02.PAD01.INV01-Inverter 2.1.1 - 3005.63kW - 2500.00kW,2096.05.005.BLK02.PAD02.INV01-Inverter 2.2.1 - 3018.75kW - 2500.00kW,2096.05.005.BLK02.PAD03.INV01-Inverter 2.3.1 - 3003.75kW - 2500.00kW,2096.05.005.BLK02.PAD04.INV01-Inverter 2.4.1 - 3003.75kW - 2500.00kW,2096.05.005.BLK02.PAD05.INV01-Inverter 2.5.1 - 3020.63kW - 2500.00kW,2096.05.005.BLK02.PAD06.INV01-Inverter 2.6.1 - 3020.63kW - 2500.00kW,2096.05.005.BLK02.PAD07.INV01-Inverter 2.7.1 - 3020.63kW - 2500.00kW,POA,Meter Power
Timestamp,,,,,,,,,,,,,,,,
2022-09-01 04:00:00+00:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.227201,-60.517791
2022-09-01 04:05:00+00:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.246880,-60.313896
2022-09-01 04:10:00+00:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.328844,-62.808157
2022-09-01 04:15:00+00:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.191575,-59.798205
2022-09-01 04:20:00+00:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.131532,-61.095226
2022-09-01 04:25:00+00:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.227195,-62.495002
2022-09-01 04:30:00+00:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.239155,-59.289884
2022-09-01 04:35:00+00:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.205524,-61.266349
2022-09-01 04:40:00+00:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.137268,-61.546576


In [10]:
inv_names = []

df = df.set_index([df.index])
df.index = pd.to_datetime(df.index)
df = df[~df.index.duplicated(keep='first')]
_tz = plant_tz(plant)
df.index = df.index.tz_convert(_tz)

''' Loading inv power and poa into dataframe '''

inv_power = df.filter(regex = 'INV')
all_power = df.fillna(0)

all_power.loc[:,'POA'] = df.filter(regex = 'POA').mean(axis = 1) #Manual


''' Parsing inverter metadata from the inverter name '''

inv_names = inv_power.columns.str.split('-').str[0].tolist()
n = len(inv_power.columns.str.split('-').tolist()[0])

rated_dc = []
rated_dc = [re.findall(r"[-+]?(?:\d*\.\d+|\d+)",i) for i in inv_power.columns.str.split('-').str[n-2].tolist()]
rated_dc = list(itertools.chain(*rated_dc))
rated_dc = [float(i) for i in rated_dc]


rated_ac = []
rated_ac = [re.findall(r"[-+]?(?:\d*\.\d+|\d+)",i) for i in inv_power.columns.str.split('-').str[n-1].tolist()]

rated_ac = list(itertools.chain(*rated_ac))
rated_ac = [float(i) for i in rated_ac]
# Add Inverter names not tags
''' Creating metadata table '''
metadata = pd.DataFrame({'Inv_name':inv_names,
                        'Rated_DC_KW':rated_dc,
                        'Rated_AC_KW':rated_ac})

In [ ]:
''' Assuming max_power as the expected inverter power '''

all_power.loc[:,'inv_model_power'] = inv_power.max(axis = 1)
inv_power.loc[:,'inv_model_power'] = inv_power.max(axis = 1)
inv_power.loc[:,'POA'] = all_power.loc[:,'POA']

''' Dataframe to store all lost energy events '''
final_df = pd.DataFrame()

''' Finding the min data resolution '''
# data resolution
res = df.index.to_series().diff().dt.total_seconds().fillna(0)[1]

''' Looping through every inverter and look for underperformance events '''
for i in range(len(inv_names)):
    norm = (metadata['Rated_DC_KW'].iloc[i]/metadata['Rated_DC_KW'].max())
    Inv_Limit = metadata['Rated_AC_KW'].iloc[i]    
    inv1 = all_power.loc[(all_power.iloc[:,i] < (0.01*metadata['Rated_AC_KW'].iloc[i]))]#&(all_power.loc[:,'POA']>0)&(all_power.loc[:,'Meter_Power']>(0.1 * POI_Limit))]

    if len(inv1) == 0:
        continue
    
    """select column which contains the inverter name and energy delivered"""
    inv1.iloc[:,i] = inv1['inv_model_power'] * norm
    inv1.iloc[:,i][(inv1.loc[:,'POA']<100)|(inv1.loc[:,'inv_model_power']<(0.01 * Inv_Limit))|(inv1.loc[:,'inv_model_power']>(0.99 * Inv_Limit))] = 0
    inv = inv1.iloc[:,i]
    
    a = inv.index.to_series().diff().dt.total_seconds().fillna(res)
    b = a[a!=res]
    temp_df = pd.DataFrame(columns = ["Start_Date","End_Date","Equipment_Name","Loss_Category"])
    temp_df.loc[0,'Start_Date'] = str(a.index[0])
    
    for j in range(len(b)):
        row_index = a.index.get_loc(b.index[j])
        t_1 = row_index - 1
        temp_df.loc[j,'End_Date'] = str(a.index[t_1])
        temp_df.loc[j+1,'Start_Date'] = str(a.index[row_index])
    temp_df.loc[len(b),'End_Date'] = str(a.index[-1])    
    
    for j in range(len(temp_df)):
        temp_df.loc[j,'Energy_loss_MWh'] = inv[temp_df.loc[j,'Start_Date']:temp_df.loc[j,'End_Date']].sum()/(1000*(60*60/res))
        
    temp_df['Equipment_Name'] = metadata['Inv_name'].iloc[i]

    final_df = pd.concat([final_df,temp_df])